In [1]:
# %load first_cell.py
%reload_ext autoreload
%autoreload 2
from pathlib import Path
home = str(Path.home())

import sys
sys.path = sys.path + [f'{home}/.conda/envs/norm_env/lib/python37.zip', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/lib-dynload', 
                       f'{home}/.conda/envs/norm_env/lib/python3.7/site-packages']
sys.prefix = '/home/soniame/.conda/envs/norm_env'

from paths import RAW_PATH, TREAT_PATH, OUTPUT_PATH, FIGURES_PATH

from copy import deepcopy
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
import yaml
import matplotlib.pyplot as plt 
import datetime

import warnings
warnings.filterwarnings('ignore')

# Plotting
import plotly
import plotly.graph_objs as go
import cufflinks as cf
plotly.offline.init_notebook_mode(connected=True)

def iplottitle(title, width=40):
    return '<br>'.join(textwrap.wrap(title, width))
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
theme_set(theme_linedraw()) # default theme

# Setting cufflinks
import textwrap
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import yaml
from jinja2 import Template
cf.themes.THEMES['custom'] = yaml.load(open('cufflinks_template.yaml', 'r'))

# Manipulation 
from siuba import *

In [2]:
from src import utils
from datetime import datetime
conn = utils.connect_athena(path='../configs/athena.yaml')

In [14]:
df = pd.read_sql_query("""
select 
    *,
    date_parse(concat('2020', ' ', cast(month as varchar), ' ', cast(day as varchar)), '%Y %m %e')
from spd_sdv_waze_corona.prod_daily_daily_index
where country_iso_code = 'MX'
""", conn)

In [16]:
print(df.shape)
df.columns

(4044, 19)


Index(['last_updated_utc', 'region_slug', 'region_name', 'country_name',
       'country_iso_code', 'country_idb_code', 'region_type', 'population',
       'timezone', 'month', 'day', 'dow', 'observed', 'expected_2020',
       'ratio_20', 'tci', 'dashboard', 'region_shapefile_wkt', 'date'],
      dtype='object')

In [19]:
df['year'] = 2020
df['year'][df.month <= 2] = 2021
df['date'] = pd.to_datetime(df[['year','month','day']]
                            .astype(str).apply(' '.join, 1), format='%Y %m %d')
df['datetime'] = df['date'].astype(str).apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))

In [23]:
# Equivale a athena spd_sdv_waze_corona.prod_daily_daily_index
#df = pd.read_csv('~/shared/spd-sdv-omitnik-waze/corona/prod/private/daily/write_index/write_index_daily.csv', sep = "|")
#df = df[df.month != 'TRUE']
#print(df.shape)

(4044, 21)


In [24]:
print(min(df.date))
print(max(df.date))
df.describe()

2020-03-09 00:00:00
2021-02-08 00:00:00


,month,day,dow,observed,expected_2020,ratio_20,tci,year
count,4044.000000,4044.000000,4044.000000,4.044000e+03,4.044000e+03,4044.000000,4044.000000,4044.000000
mean,6.878338,15.821958,3.991098,5.038475e+07,5.923907e+07,0.820834,-17.916585,2020.115727
std,3.353084,8.846840,2.003934,1.357201e+08,1.207522e+08,0.669419,66.941885,0.319937
min,1.000000,1.000000,1.000000,1.299180e+05,6.317860e+05,0.079478,-92.052232,2020.000000
25%,4.000000,8.000000,2.000000,2.493400e+06,4.199193e+06,0.381671,-61.832936,2020.000000
50%,7.000000,16.000000,4.000000,5.564834e+06,1.103633e+07,0.671789,-32.821087,2020.000000
75%,10.000000,23.000000,6.000000,1.780378e+07,3.278052e+07,0.994350,-0.564970,2020.000000
max,12.000000,31.000000,7.000000,1.252923e+09,5.187512e+08,7.468349,646.834889,2021.000000


In [152]:
print(len(df['region_slug'].unique()))
df['country_name'].unique()

108


array(['Brazil', 'Colombia', 'Panama', 'Chile', 'Mexico', 'Ecuador',
       'Argentina', 'Barbados', 'Bolivia', 'Nicaragua', 'El Salvador',
       'Peru', 'Honduras', 'Dominican Republic', 'Uruguay', 'Jamaica',
       'Trinidad and Tobago', 'Paraguay', 'Costa Rica', 'Guatemala'],
      dtype=object)

Days observed per region_slug

In [35]:
df \
  .siu_group_by('region_slug', "country_name") \
  .siu_summarize(nc = _.tci.count(),
                 tci = _.tci.mean(),
                 tci_min = _.tci.min(),
                 tci_max = _.tci.max(),
                 dat_min = _.date.min(),
                 dat_max = _.date.max()
                 ) \
  .siu_ungroup()

,region_slug,country_name,nc,tci,tci_min,tci_max,dat_min,dat_max
0,aguascalientes,Mexico,337,-19.379488,-89.772846,295.093715,2020-03-09,2021-02-08
1,country_mexico,Mexico,337,3.992295,-83.873186,236.146628,2020-03-09,2021-02-08
2,guadalajara,Mexico,337,-34.205306,-88.839614,101.185662,2020-03-09,2021-02-08
3,leon,Mexico,337,-15.228397,-90.705502,171.461936,2020-03-09,2021-02-08
4,merida,Mexico,337,-44.380218,-85.996939,69.881371,2020-03-09,2021-02-08
5,mexicocity,Mexico,337,-40.625139,-92.052232,80.020119,2020-03-09,2021-02-08
6,monterrey,Mexico,337,-43.346575,-89.642462,234.132600,2020-03-09,2021-02-08
7,pueble,Mexico,337,-30.052805,-81.048294,90.490344,2020-03-09,2021-02-08
8,queretaro,Mexico,337,-44.634923,-89.328917,105.990360,2020-03-09,2021-02-08
9,sanluispotosi,Mexico,337,-7.919481,-91.638874,457.361712,2020-03-09,2021-02-08


TCI observations per date 

In [36]:
tab = df \
      .siu_group_by('region_slug', "country_name", "date") \
      .siu_summarize(nc = _.tci.count(),
                     tci = _.tci.mean(),
                     tci_min = _.tci.min(),
                     tci_max = _.tci.max()
                     ) \
      .siu_ungroup()
tab.head()

,region_slug,country_name,date,nc,tci,tci_min,tci_max
0,aguascalientes,Mexico,2020-03-09,1,-42.103947,-42.103947,-42.103947
1,aguascalientes,Mexico,2020-03-10,1,3.740975,3.740975,3.740975
2,aguascalientes,Mexico,2020-03-11,1,8.477060,8.477060,8.477060
3,aguascalientes,Mexico,2020-03-12,1,13.016856,13.016856,13.016856
4,aguascalientes,Mexico,2020-03-13,1,-6.457111,-6.457111,-6.457111


In [37]:
print(tab.shape)
tab.siu_filter(_.nc > 1).shape

(4044, 7)


(0, 7)

In [39]:
#tab.siu_filter(_.nc > 1).head()

### TCI time serie per region

In [46]:
df_sel=df[['date', 'region_slug', 'tci', 'observed', 'expected_2020', 'ratio_20', 'population']]
df_sel.head()

,date,region_slug,tci,observed,expected_2020,ratio_20,population
0,2020-11-10,mexicocity,-25.312853,140246434,187778540.0,0.746871,22381714
1,2020-09-15,mexicocity,-29.258810,132836773,187778540.0,0.707412,22381714
2,2021-01-26,mexicocity,-34.923075,122200499,187778540.0,0.650769,22381714
3,2021-01-05,mexicocity,-21.142748,148076996,187778540.0,0.788573,22381714
4,2020-10-14,guadalajara,-36.472050,27889031,43900411.0,0.635279,4793169


In [99]:
df_sel = (df
 >> select('date', 'dow', 'region_slug', 'tci', 'observed', 'expected_2020', 'ratio_20', 'population')
 >> arrange(_.region_slug, _.date)
 >> mutate(observed_log = _.observed)
 >> group_by('region_slug')
 >> mutate(observed_1day = _.observed.shift(1),
           observed_7day = _.observed.shift(7))
 >> ungroup()
 >> group_by('region_slug', 'dow')    
 >> mutate(observed_1dow = _.observed.shift(1))          
 >> ungroup()          
 >> mutate(diff_1day = _.observed - _.observed_1day,
           diff_7day = _.observed - _.observed_7day)         
)

In [100]:
df_sel.head(9)

,date,dow,region_slug,tci,observed,expected_2020,ratio_20,population,observed_log,observed_1day,observed_7day,observed_1dow,diff_1day,diff_7day
0,2020-03-09,1,aguascalientes,-42.103947,1263784,2182850.0,0.578961,911397,1263784,NaN,NaN,NaN,NaN,NaN
1,2020-03-10,2,aguascalientes,3.740975,2865805,2762462.0,1.037410,911397,2865805,1263784.0,NaN,NaN,1602021.0,NaN
2,2020-03-11,3,aguascalientes,8.477060,3073397,2833223.0,1.084771,911397,3073397,2865805.0,NaN,NaN,207592.0,NaN
3,2020-03-12,4,aguascalientes,13.016856,3009640,2663001.0,1.130169,911397,3009640,3073397.0,NaN,NaN,-63757.0,NaN
4,2020-03-13,5,aguascalientes,-6.457111,3196426,3417070.0,0.935429,911397,3196426,3009640.0,NaN,NaN,186786.0,NaN
5,2020-03-14,6,aguascalientes,14.228133,3623811,3172433.0,1.142281,911397,3623811,3196426.0,NaN,NaN,427385.0,NaN
6,2020-03-15,7,aguascalientes,-5.568909,1381338,1462800.0,0.944311,911397,1381338,3623811.0,NaN,NaN,-2242473.0,NaN
7,2020-03-16,1,aguascalientes,-56.431179,951042,2182850.0,0.435688,911397,951042,1381338.0,1263784.0,1263784.0,-430296.0,-312742.0
8,2020-03-17,2,aguascalientes,-32.865683,1854560,2762462.0,0.671343,911397,1854560,951042.0,2865805.0,2865805.0,903518.0,-1011245.0


In [94]:
df_sel[['date', 'observed', 'region_slug']] \
    .pivot(index='date', columns='region_slug', values='observed') \
    .iplot(
    #theme='custom',
    yTitle='TCI',
    title='TCI Difference One day',
)

In [90]:
df_sel[['date', 'diff_1day', 'region_slug']] \
    .pivot(index='date', columns='region_slug', values='diff_1day') \
    .iplot(
    #theme='custom',
    yTitle='TCI',
    title='TCI Difference One day',
)

In [91]:
df_sel[['date', 'diff_7day', 'region_slug']] \
    .pivot(index='date', columns='region_slug', values='diff_7day') \
    .iplot(
    #theme='custom',
    yTitle='TCI',
    title='TCI Difference One day',
)

In [92]:
#(ggplot(tt,
#        aes('date', 'diff', color='factor(region_slug)'))
# + geom_line()
# + geom_point()
# + theme(axis_text_x=element_text(angle=90)))